In [1]:
import pandas as pd
import numpy as np
import sys
import math 
import seaborn as sea
import datetime as dt
from datetime import timedelta
from operator import itemgetter
from functools import partial

In [2]:
def triangulo(var, pontos):
     a = pontos[0]
     b = pontos[1]
     c = pontos[2]
     result = 0
     if var >= a and var < b:
        result = (var - a) / float(b - a)
     elif var > b and var <= c:
        result = (c - var) / float(c - b)
     elif var == b: 
        result = 1
     return result

In [3]:
def trapezio(var,pontos):
    a = pontos[0]
    b = pontos[1]
    c = pontos[2]
    d = pontos[3]
    result = 0
    if var >= a and var < b:
        result = triangulo(var, [a, b, b])
    elif var >= b and var <= c:
        result = 1
    elif var > c and var <= d:
        result = triangulo(var, [c, c, d])
    return result

In [4]:
def defuzzyfication(indice):
    ##Indice: 0- Queda Brusca, 1 - Queda Moderada, 2- Platô, 3- Subida Moderada, 4-Subida Brusca
    Index = 0
    if indice == 0:
        Index = 0
    elif indice == 1:
        Index = 1
    elif indice == 2:
        Index = 2
    elif indice == 3:
        Index = 3
    elif indice == 4:
        Index = 4
    return Index

In [5]:
def fuzzifyErrorsPos(param, media):
    return trapezio(param, [(media + ((media * 26) / 100)), (media + ((media * 50) / 100)), (media + ((media * 80) / 100)), (media + ((media * 80) / 100))])

def fuzzifyparamPos(param, media):
    return triangulo(param, [(media + ((media * 8) / 100)), (media + ((media * 18) / 100)), (media + ((media * 28) / 100))])

def fuzzifyparamZ(param, media):
    return triangulo(param, [(media - ((media * 10) / 100)), media, (media + ((media * 10) / 100))])

def fuzzifyparamNeg(param, media):
    return triangulo(param, [(media - ((media * 28) / 100)), (media - ((media * 18) / 100)), (media - ((media * 8) / 100))])

def fuzzifyparamsNeg(param, media):
    return trapezio(param, [0, 0, (media - ((media * 50) / 100)), (media - ((media * 26) / 100))])

In [6]:
def verifRegras(param, media):
    regras = [0] * 5
    
    fuzzifyparamsNega = fuzzifyparamsNeg(param, media)
    fuzzifyparamNega = fuzzifyparamNeg(param, media)
    fuzzifyparamZe = fuzzifyparamZ(param, media)
    fuzzifyparamPosi = fuzzifyparamPos(param, media)
    fuzzifyparamsPosi = fuzzifyparamPos(param, media)
    
    regras[0] = fuzzifyparamNega
    regras[1] = fuzzifyparamNega
    regras[2] = fuzzifyparamZe    
    regras[3] = fuzzifyparamPosi    
    regras[4] = fuzzifyparamsPosi
    
    return regras

In [7]:
def getPercentage(rules):
    indice = np.argmax(rules)
    percent = (rules[indice] * 100)
    return percent

In [8]:
def getCentroid(media,rules,incl):
    
    centroid = 0
    indice = np.argmax(rules)
    if indice == 0:
        if rules[indice]>95:
            return 0
        elif rules[indice]>50:
            return (media - ((media * 50) / 100)) * rules[indice]
        else:
            return (media - ((media * 26) / 100)) * rules[indice]
    elif indice == 1:
        if rules[indice]>95:
            return (media - ((media * 18) / 100)) * rules[indice]
        else:
            if incl < (media - ((media * 18) / 100)):
                return (media - ((media * 28) / 100)) * rules[indice]
            else:
                return (media - ((media * 8) / 100)) * rules[indice]
    elif indice == 2:
        if rules[indice]>95:
            media * rules[indice]
        else:
            if incl < media  :
                return (media - ((media * 10) / 100)) * rules[indice]
            else:
                return (media + ((media * 10) / 100)) * rules[indice]
            
    elif indice == 3:
        if rules[indice]>95:
            return (media + ((media * 18) / 100)) * rules[indice]
        else:
            if incl < (media + ((media * 18) / 100)):
                return (media + ((media * 8) / 100)) * rules[indice]
            else:
                return (media + ((media * 28) / 100)) * rules[indice]        
    elif indice == 4:
        if rules[indice]>95:
            return (media + ((media * 80) / 100)) * rules[indice]
        elif rules[indice]>50:
            return (media + ((media * 50) / 100)) * rules[indice]
        else:
            return (media - ((media * 26) / 100)) * rules[indice]
    return centroid